In [ ]:
#System Modules
import os.path

# Deep Learning Modules
from tensorboardX import SummaryWriter
from torch.autograd import Variable
from torch.utils.data import Dataset, ConcatDataset
from torchvision.utils import make_grid
from torch.nn import *
# User Defined Modules
import specs
from data.Img_dataset import *
from models import UNet

from serde import *
from utils.augmentation import *
from Training import Training
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
cfg_path = './config.json'
#Uncomment below line if you want to delete an experiment and reuse the same experiment name
#delete_experiment('UNet_SGD')
params = create_experiment('UNet_SGD')
cfg_path=params["cfg_path"]

In [ ]:
# Train Set

augmentation_operation = Compose([LeftRightFlip(cfg_path), Rotate(cfg_path),
                                  ColRec(cfg_path),GaussianNoise(cfg_path),
                                  ZoomIn(cfg_path),Invert(cfg_path)
                                 ], cfg_path)
train_dataset = ConcatDataset([
        Img_dataset(dataset_name='TrainSet1000',size=1000,cfg_path=cfg_path,augmentation=augmentation_operation),
        Img_dataset(dataset_name='True_Negative_Dataset_Train',size=300,  cfg_path=cfg_path),

    ])


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=1,
                                               shuffle=True, num_workers=4)

# Test Set
test_dataset =  Img_dataset(dataset_name='TestSet20',size=20,cfg_path=cfg_path,mode=Mode.TEST, seed=5)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=1,
                                              shuffle=False, num_workers=4)



In [ ]:
#Initialize trainer
#cfg_path=params['cfg_path']
trainer=Training(cfg_path)

In [ ]:
#Defime model parameters
optimiser_params={'lr':0.01}
lane_to_nolane_weight_ratio=4
trainer.setup_model(model=UNet,
                    optimiser=torch.optim.SGD,optimiser_params=optimiser_params,
                    loss_function=CrossEntropyLoss,weight_ratio=lane_to_nolane_weight_ratio)

In [ ]:
#Execute Training
trainer.execute_training(train_loader,test_loader,num_epochs=25)